In [4]:
import pandas as pd
import numpy as np
from google.colab import files
uploaded = files.upload()
upload = files.upload()

Saving p2_train.csv to p2_train.csv


Saving p2_test.csv to p2_test.csv


In [88]:
import io
q_n_a = pd.read_csv(io.BytesIO(uploaded['p2_train.csv']))
q_n_a_test = pd.read_csv(io.BytesIO(upload['p2_test.csv']))
train_features=[]

In [89]:
q_n_a.head

<bound method NDFrame.head of      thread_id  ...        type
0       2rnfn0  ...    answered
1       3htb3d  ...    attacked
2       2ffph1  ...    attacked
3       2qix6r  ...    answered
4       2f7hxq  ...    answered
...        ...  ...         ...
1635    2rieng  ...    answered
1636    27q9ys  ...  irrelevant
1637    25gvjj  ...  irrelevant
1638    3g1yem  ...  irrelevant
1639    3fwi59  ...    attacked

[1640 rows x 10 columns]>

In [90]:
q_n_a.dtypes

thread_id       object
question_id     object
response_id     object
no_turn_q_id    object
quoted_q_id     object
precedent       object
question        object
subsequent      object
response        object
type            object
dtype: object

In [91]:
Q = q_n_a.loc[:, ['question']].values
A = q_n_a.loc[:, ['response']].values
Q_test = q_n_a_test.loc[:, ['question']].values
A_test = q_n_a_test.loc[:, ['response']].values


Label = q_n_a.loc[:,['type']].values
Label_test = q_n_a_test.loc[:,['type']].values

In [92]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

#nlp = spacy.load('en_core_web_md')


In [93]:
question_emb=[]
question_emb_test=[]
for line in Q:
   #question_emb.append([sbert_model.encode(line[0])])
  doc = nlp(line[0])
  question_emb.append(doc.vector)

for line in Q_test:
  doc = nlp(line[0])
  question_emb_test.append(doc.vector)


In [94]:
answer_emb_test=[]
answer_emb=[]
for line in A:
    doc = nlp(line[0])
    answer_emb.append(doc.vector)
for line in A_test:
   doc = nlp(line[0])
   answer_emb_test.append(doc.vector)


In [95]:
train_features=[]

for q_emb,a_emb in zip(question_emb, answer_emb):
  train_features.append(q_emb+a_emb)

In [96]:
test_features=[]
for q_emb,a_emb in zip(question_emb_test, answer_emb_test):
    test_features.append(q_emb+a_emb)
  # np.concatenate((q_emb,a_emb),axis=None)
  # test_features.append(np.concatenate((q_emb,a_emb),axis=None))

In [97]:
#pos tagging

In [98]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [99]:
train_tagged_q=[]
train_tagged_a=[]
for line in Q:
    wordsList = nltk.word_tokenize(line[0])  
    train_tagged_q.append(nltk.pos_tag(wordsList) )
for line in A:
    wordsList = nltk.word_tokenize(line[0])  
    train_tagged_a.append(nltk.pos_tag(wordsList) )

In [100]:
test_tagged_q=[]
test_tagged_a=[]
for line in Q_test:
    wordsList = nltk.word_tokenize(line[0])  
    test_tagged_q.append(nltk.pos_tag(wordsList) ) 
for line in A_test:
    wordsList = nltk.word_tokenize(line[0])  
    test_tagged_a.append(nltk.pos_tag(wordsList) )

In [101]:
from sklearn.feature_extraction import DictVectorizer

vectoriser=DictVectorizer(sparse= False )
train_vector_q=[]
train_vector_a=[]
for line in train_tagged_q:
  line_vector=[]
  for word,tag in line: 
    line_vector.append(tag) 
  train_vector_q.append(line_vector)

for line in train_tagged_a:
  line_vector=[]
  for word,tag in line: 
    line_vector.append(tag) 
  train_vector_a.append(line_vector)


In [102]:
test_vector_q=[]
test_vector_a=[]
for line in test_tagged_q:
  line_vector=[]
  for word,tag in line: 
    line_vector.append(tag) 
  test_vector_q.append(line_vector)

for line in test_tagged_a:
  line_vector=[]
  for word,tag in line: 
    line_vector.append(tag) 
  test_vector_a.append(line_vector)


In [103]:
feature_train_vector=[]
for line_q,line_a in zip(train_vector_q,train_vector_a):
  feature_train_vector.append(line_q+line_a)

In [104]:
feature_test_vector=[]
for line_q,line_a in zip(test_vector_q,test_vector_a):
  feature_test_vector.append(line_q+line_a)

In [105]:
print((train_vector_q))
print((feature_train_vector))
print((feature_test_vector))

[['MD', 'PDT', 'DT', 'NNS', 'IN', 'DT', 'NN', 'NN', 'IN', '.'], ['RB', ',', 'VB', 'PRP', 'IN', 'PRP', 'VBP', 'JJ', ',', 'CC', 'WP', 'PRP', 'VBP', 'VBG', 'VBZ', 'IN', 'DT', 'NN', 'VBG', 'NN', 'JJ', 'IN', 'DT', 'NNP', 'CC', 'NN', ':', 'CC', 'JJ', 'JJ', 'NNP', 'CC', 'NN', ':', 'CC', 'VBZ', 'VBG', 'DT', 'JJ', 'JJ', 'NN', '.'], ['WP', 'IN', 'DT', 'DT', 'NN', '.'], ['CC', 'NNS', 'VBP', 'JJR', 'NNS', 'IN', 'NNS', ',', 'VBZ', 'CD', 'TO', 'CD', 'NN', 'JJR', 'NNS', 'IN', 'DT', 'IN', 'CD', ',', 'DT', 'JJ', 'RBR', 'JJ', 'TO', 'VB', 'NNS', 'IN', 'DT', 'JJ', ',', 'FW', ':', 'RB', 'WRB', 'RB', 'VB', 'CD', 'IN', 'DT', 'NNS', '.'], ['WRB', 'VBZ', 'IN', 'NNP', 'POS', 'NN', '.'], ['RB', 'PRP', 'VBZ', 'RB', 'RBR', 'RB', 'JJ', 'TO', 'DT', 'NN', 'IN', 'NNS', 'TO', 'VB', 'PRP$', 'NN', 'IN', 'JJR', 'NNS', 'IN', 'JJR', 'NNS', '.'], ['IN', 'VBG', 'DT', 'NN', 'NN', ',', 'DT', 'NN', 'IN', 'NN', 'VBD', 'VBG', 'IN', '(', 'NNP', 'RB', 'IN', 'DT', 'NN', 'VBN', 'NNP', '.', ')'], ['CC', 'DT', 'VBP', 'RB', 'RB', 'JJ', '

In [106]:
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer()
transfomed_label = encoder.fit_transform(["","CC","CD","DT","EX","FW","IN","JJ","JJR","JJS","LS","MD","NN","NNS","NNP","NNPS","PDT","POS","PRP","PRP$","RB","RBR","RBS","RP","SYM","TO","UH","VB","VBD","VBG","VBN","VBP","VBZ","WDT","WP","WP$","WRB"])
labels = encoder.classes_
mappings = {}
for index, label in zip(range(len(labels)), labels):
  mappings[label]=index
pos_feature=[]

for sent in range(len(feature_train_vector)):
  res_temp=np.zeros(len(mappings))
  for j in range(len(feature_train_vector[sent])):
    if len(feature_train_vector[sent])>1 and feature_train_vector[sent][j] in mappings.keys():
      res_temp+=(transfomed_label[mappings[feature_train_vector[sent][j]]])
    res_temp+=transfomed_label[mappings[""]]
    res_temp=np.asarray(res_temp)
  pos_feature.append(res_temp)

In [107]:
pos_feature_test=[]

for sent in range(len(feature_test_vector)):
  res_temp=np.zeros(len(mappings))
  for j in range(len(feature_test_vector[sent])):
    if len(feature_test_vector[sent])>1 and feature_test_vector[sent][j] in mappings.keys():
      res_temp+=(transfomed_label[mappings[feature_test_vector[sent][j]]])
    res_temp+=transfomed_label[mappings[""]]
    res_temp=np.asarray(res_temp)
  pos_feature_test.append(res_temp)

In [108]:
#sentiment analysis

In [109]:
#takes around 30 mins
# !wget https://nlp.stanford.edu/software/stanford-corenlp-full-2018-10-05.zip https://nlp.stanford.edu/software/stanford-english-corenlp-2018-10-05-models.jar
# !unzip stanford-corenlp-full-2018-10-05.zip
# !mv stanford-english-corenlp-2018-10-05-models.jar stanford-corenlp-full-2018-10-05
# !cd stanford-corenlp-full-2018-10-05
# !java -mx6g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -timeout 5000

# !pip install pycorenlp


In [110]:
# !cd stanford-corenlp-full-2018-10-05
# !java -cp "*" -Xmx2g edu.stanford.nlp.pipeline.StanfordCoreNLP

# from pycorenlp import StanfordCoreNLP
# nlp = StanfordCoreNLP('http://localhost:9000')
# train_sentiment_q=[]
# train_sentiment_a=[]
# for line in Q:
#     result = nlp.annotate(line[0],properties={'annotators': 'sentiment','outputFormat': 'json', })
#     print(result["sentences"]["sentimentValue"])
#     train_sentiment_q.append(nltk.pos_tag(wordsList) )
# for line in A:
#     wordsList = nltk.word_tokenize(line[0])  
#     train_tagged_a.append(nltk.pos_tag(wordsList) )
# result = nlp.annotate(text,
#                    properties={
#                        'annotators': 'sentiment, ner, pos',
#                        'outputFormat': 'json',
#                        'timeout': 1000,
                  #  })


In [119]:
from textblob import TextBlob
opinion = TextBlob("EliteDataScience.com is bad.")
train_sentiment_q=[]
train_sentiment_a=[]
test_sentiment_q=[]
test_sentiment_a=[]
for line in Q:
     result = TextBlob(line[0]).sentiment.polarity
     train_sentiment_q.append([result])

for line in A:
     result = TextBlob(line[0]).sentiment.polarity
     train_sentiment_a.append([result])

for line in Q_test:
     result = TextBlob(line[0]).sentiment.polarity
     test_sentiment_q.append([result])

for line in A_test:
     result = TextBlob(line[0]).sentiment.polarity
     test_sentiment_a.append([result])

In [120]:
train_sentiment_all=[]
for line_q,line_a in zip(train_sentiment_q,train_sentiment_a):
  train_sentiment_all.append(line_q+line_a)

In [121]:
test_sentiment_all=[]
for line_q,line_a in zip(test_sentiment_q,test_sentiment_a):
  test_sentiment_all.append(line_q+line_a)

In [122]:
# print(train_features.shape)
pos_feature=np.vstack(pos_feature)
print(pos_feature.shape)
pos_feature_test=np.vstack(pos_feature_test)
print(pos_feature_test.shape)


(1640, 37)
(410, 37)


In [ ]:
#summarisation and cosine similarity 

In [138]:
nltk.download('stopwords')

from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
  
sw = stopwords.words('english')  
l1 =[];l2 =[] 
Q_set=[]
A_set=[]
train_cosine_feature=[]
for line in Q:
  q = word_tokenize(line[0]) 
  Q_set.append([w for w in q if not w in sw])  
 
for line in A:
  a =  word_tokenize(line[0]) 
  A_set.append([w for w in a if not w in sw])
  
for q_set,a_set in zip(Q_set,A_set):
  all_set = q_set + a_set  
  for w in all_set: 
    if w in q_set: l1.append(1) # create a vector 
    else: l1.append(0) 
    if w in a_set: l2.append(1) 
    else: l2.append(0) 
  c = 0
  
  for i in range(len(all_set)): 
        c+= l1[i]*l2[i] 
  cosine = c / float((sum(l1)*sum(l2))**0.5) 
  train_cosine_feature.append([cosine])
train_cosine_feature=np.vstack(train_cosine_feature)
print(train_cosine_feature.shape)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(1640, 1)


In [137]:
sw = stopwords.words('english')  
l1 =[];l2 =[] 
Q_set=[]
A_set=[]
test_cosine_feature=[]
for line in Q_test:
  q = word_tokenize(line[0]) 
  Q_set.append([w for w in q if not w in sw])  
 
for line in A_test:
  a =  word_tokenize(line[0]) 
  A_set.append([w for w in a if not w in sw])
  
for q_set,a_set in zip(Q_set,A_set):
  all_set = q_set + a_set  
  for w in all_set: 
    if w in q_set: l1.append(1) # create a vector 
    else: l1.append(0) 
    if w in a_set: l2.append(1) 
    else: l2.append(0) 
  c = 0
  
  for i in range(len(all_set)): 
        c+= l1[i]*l2[i] 
  cosine = c / float((sum(l1)*sum(l2))**0.5) 
  test_cosine_feature.append([cosine])
test_cosine_feature=np.vstack(test_cosine_feature)
print(test_cosine_feature.shape)

(410, 1)


In [ ]:
# combine features

In [141]:
combined_train=[]
for emb,pos,sent,cos in zip(train_features,pos_feature,train_sentiment_all,train_cosine_feature):
  combined_train.append(np.concatenate(
      ((np.concatenate(
          (np.concatenate(
              (emb,pos), axis=None)
          ,sent),axis=None))
      ,cos),axis=None))

combined_train = np.vstack(combined_train)
combined_train.shape

(1640, 136)

In [143]:
combined_test=[]
for emb,pos,sent,cos in zip(test_features,pos_feature_test,test_sentiment_all,train_cosine_feature):
  combined_test.append(np.concatenate(
      ((np.concatenate(
          (np.concatenate(
              (emb,pos), axis=None)
          ,sent),axis=None))
      ,cos),axis=None))
combined_test=np.vstack(combined_test)
combined_test.shape

(410, 136)

In [144]:
#evaluation : classifier KNN
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors=5)
Label=np.asarray(Label)
classifier.fit(combined_train, Label)
y_pred = classifier.predict(combined_test)

print(confusion_matrix(Label_test,y_pred))
print(classification_report(Label_test,y_pred))
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

print("accuracy_score : ",accuracy_score(Label_test,y_pred))
print("precision_score : ",precision_score(Label_test,y_pred, average='weighted'))
print("recall_score : ",recall_score(Label_test,y_pred, average='weighted'))
print("f1_score : ",f1_score(Label_test,y_pred, average='weighted'))

[[  0  12   1   0]
 [  3 275  29  13]
 [  0  33   3   3]
 [  0  33   4   1]]
              precision    recall  f1-score   support

      agreed       0.00      0.00      0.00        13
    answered       0.78      0.86      0.82       320
    attacked       0.08      0.08      0.08        39
  irrelevant       0.06      0.03      0.04        38

    accuracy                           0.68       410
   macro avg       0.23      0.24      0.23       410
weighted avg       0.62      0.68      0.65       410

accuracy_score :  0.6804878048780488
precision_score :  0.62119327068868
recall_score :  0.6804878048780488
f1_score :  0.6487228471543905


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


In [145]:
#build model: classifier svm
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')

svclassifier.fit(combined_train, Label)
y_pred = svclassifier.predict(combined_test)




/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [146]:
#evaluate model: classifier svm
print(confusion_matrix(Label_test,y_pred))
print(classification_report(Label_test,y_pred))

print("accuracy_score : ",accuracy_score(Label_test,y_pred))
print("precision_score : ",precision_score(Label_test,y_pred, average='weighted'))
print("recall_score : ",recall_score(Label_test,y_pred, average='weighted'))
print("f1_score : ",f1_score(Label_test,y_pred, average='weighted'))

[[  2  10   0   1]
 [  8 307   1   4]
 [  0  37   0   2]
 [  1  32   0   5]]
              precision    recall  f1-score   support

      agreed       0.18      0.15      0.17        13
    answered       0.80      0.96      0.87       320
    attacked       0.00      0.00      0.00        39
  irrelevant       0.42      0.13      0.20        38

    accuracy                           0.77       410
   macro avg       0.35      0.31      0.31       410
weighted avg       0.67      0.77      0.70       410

accuracy_score :  0.7658536585365854
precision_score :  0.6651335163846221
recall_score :  0.7658536585365854
f1_score :  0.7026023169580139


In [147]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
random_forest_clf = RandomForestClassifier(max_depth=2, random_state=0)
random_forest_clf.fit(combined_train, Label)
y_pred = random_forest_clf.predict(combined_test)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [148]:
#evaluate model: classifier random forest
print(confusion_matrix(Label_test,y_pred))
print(classification_report(Label_test,y_pred))

print("accuracy_score : ",accuracy_score(Label_test,y_pred))
print("precision_score : ",precision_score(Label_test,y_pred, average='weighted'))
print("recall_score : ",recall_score(Label_test,y_pred, average='weighted'))
print("f1_score : ",f1_score(Label_test,y_pred, average='weighted'))

[[  0  13   0   0]
 [  0 320   0   0]
 [  0  39   0   0]
 [  0  38   0   0]]
              precision    recall  f1-score   support

      agreed       0.00      0.00      0.00        13
    answered       0.78      1.00      0.88       320
    attacked       0.00      0.00      0.00        39
  irrelevant       0.00      0.00      0.00        38

    accuracy                           0.78       410
   macro avg       0.20      0.25      0.22       410
weighted avg       0.61      0.78      0.68       410

accuracy_score :  0.7804878048780488
precision_score :  0.6091612135633552
recall_score :  0.7804878048780488
f1_score :  0.6842632809889744


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
